In [28]:
import os
import nltk
import json
import csv
import pandas as pd
from pprint import pprint
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

#pip install translate

#nltk.download()
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('vader_lexicon')
#nltk.download([
#     "names",
#     "stopwords",
#     "state_union",
#     "twitter_samples",
#     "movie_reviews",
#     "averaged_perceptron_tagger",
#     "vader_lexicon",
#     "punkt",
# ])

In [4]:
# Configs

directory = 'outputs'  # Replace with the desired directory path

In [9]:
# Utils

def getJsonFileList():
    found_files = []
    
    for filename in os.listdir(directory):
        if "reviews-" in filename and "-total" in filename:
            found_files.append(filename)
    
    return found_files


def toSentimentalAnalysis(text):
    scores = sia.polarity_scores(text)
    return scores

In [29]:
# Lendo o arquivo JSON

fileList = getJsonFileList()
reviewsToSentiment = []

def addStoreNameToList(list, store, company):
    for objeto in list:
        objeto["store"] = store
        objeto["company"] = company

    return list

for fileName in fileList:
    fileOutput = f"outputs/{fileName}"

    with open(fileOutput, 'r') as file:
        data = json.load(file)

        gReviews = data['GooglePlay']
        aReviews = data['AppleStore']
        company = data['Company']

        addStoreNameToList(gReviews, "GooglePlay", company)
        addStoreNameToList(aReviews, "AppleStore", company)

        reviewsToSentiment = gReviews + aReviews + reviewsToSentiment
    

In [31]:
# Criando sentimentação Google Reviews

def handleReviews(review):
    
    if (review['store'] == "GooglePlay"):
        contentPTBR = review['content']
        contentEN = review['ContentEN']
        score = toSentimentalAnalysis(contentEN)
        score["storeScore"] = review['score']
        score["reviewTextPTBR"] = contentPTBR
        score["reviewTextEN"] = contentEN
        score["store"] = review['store']
        score["company"] = review['company']
        
        return score
    
    if (review['store'] == "AppleStore"):
        contentPTBR = review['review']
        contentEN = review['ContentEN']
        score = toSentimentalAnalysis(contentEN)
        score["storeScore"] = review['rating']
        score["reviewTextPTBR"] = contentPTBR
        score["reviewTextEN"] = contentEN
        score["store"] = review['store']
        score["company"] = review['company']
        
        return score
    
reviewsScore = list(map(handleReviews, reviewsToSentiment))

In [33]:
# Escrever o JSON em um arquivo CSV

nome_arquivo = "outputs/sentimental-analysis-data.xlsx"

df = pd.DataFrame(reviewsScore)
df.to_excel(nome_arquivo, index=False)